# Notebook to extract hidden-states and attention heads activations from LSTM model predictions

In [24]:
import os
import glob
import torch
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from model import LSTMExtractor
from sklearn.preprocessing import StandardScaler
from numpy import linalg as la
from tokenizer import tokenize, preprocess
from data import Dictionary, Corpus
from lstm_utils import read_yaml, save_yaml, batchify_text_with_memory_size, load

In [25]:
def check_folder(path):
    """Create adequate folders if necessary."""
    try:
        if not os.path.isdir(path):
            check_folder(os.path.dirname(path))
            os.mkdir(path)
    except:
        pass

In [26]:
def transform(activations, path, name, run_index, n_layers_hidden=1, hidden_size=300):
    """activations - surprisal - entropy
    """
    assert activations.values.shape[1] == ((n_layers_hidden) * hidden_size + 2)
    indexes = [[index*hidden_size, (index+1)*hidden_size] for index in range(n_layers_hidden)]
    for order in [2]:
        matrices = []
        for index in indexes:
            matrix = activations.values[:, index[0]:index[1]]
            #with_std = True if order=='std' else False
            #scaler = StandardScaler(with_mean=True, with_std=with_std)
            #scaler.fit(matrix)
            #matrix = scaler.transform(matrix)
            if order is not None and order != 'std':
                matrix = matrix / np.mean(la.norm(matrix, ord=order, axis=1))
            matrices.append(matrix)
        matrices = np.hstack(matrices)
        print(matrices.shape)
        matrices = np.hstack([matrices, activations.values[:, -2:]])
        print(matrices.shape)
        new_data = pd.DataFrame(matrices, columns=activations.columns)
        new_path = path + '_norm-' + str(order).replace('np.', '')
        check_folder(new_path)
        new_data.to_csv(os.path.join(new_path, name + '_run{}.csv'.format(run_index + 1)), index=False)


Defining variables:

In [17]:
template = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/text_english_run*.txt' # path to text input
language = 'english'

In [5]:
name_template = 'weights_{}_embedding-size-{}_nhid-{}_nlayers-{}_dropout-{}_memory-size-{}_wiki-kristina_english.pt'
name_template = 'LSTM_embedding-size_600_nhid_300_nlayers_1_dropout_02_gutenberg_9_english.pt'

name_templates =[
#"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_Gutenberg_big_+_wiki_+_lpp_2_english.pt",
#"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_Gutenberg_big_+_wiki_2_english.pt",
#"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_2_english.pt",
#"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_2_englishless_epochs.pt",
#"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_Gutenberg_5_englishless_epochs.pt",
#"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_Gutenberg_2_englishless_epochs.pt",
#"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_Gutenberg_1_englishless_epochs.pt",
#"LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_3_english.pt",
#"LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_2_english.pt",
#"LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_1_english.pt",
    
"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_wikipedia_english.pt",
"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_3_english.pt",
"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_2_english.pt",
"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_1_english.pt",
"LSTM_embedding-size_768_nhid_300_nlayers_1_dropout_02_gutenberg_big_+_wiki_2_english.pt",

]

In [6]:
rnn_types = ['LSTM'] * len(name_templates)
ninps = ['768'] * len(name_templates)
nhids =  ['768', '768', '768', '768', '300'] * len(name_templates)
nlayers = [
    #'2', '2', '2',
    '2', '2', '2', '2', 
    '1'
]
dropouts =  ['02'] * len(name_templates)
memory_sizes = [np.inf] * len(name_templates) #, np.inf
vocab_paths = [
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_+_wiki_+_lpp_2',
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_+_wiki_2',
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_2',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/wikipedia',    
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_3',    
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_2',    
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_1',    
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_+_wiki_2',    
]
config_path_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/'
trained_model_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/'
path_to_data = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/'

In [7]:
#template = '/Users/alexpsq/Code/Parietal/data/text_english_run*.txt' # path to text input
#config_path_folder = '/Users/alexpsq/Code/Parietal/data/configs/'
#trained_model_folder = '/Users/alexpsq/Code/Parietal'
#path_to_data = '/Users/alexpsq/Code/data/stimuli-representations'
#vocab_path = '/Users/alexpsq/Code/data/'

In [8]:
infos = [
    #['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '2', 'dropout', '02', 'gutenberg_big_+_wiki_+_lpp', '2', 'english.pt'],
    #['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '2', 'dropout', '02', 'gutenberg_big_+_wiki', '2', 'english.pt'],
    #['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '2', 'dropout', '02', 'gutenberg_big', '2', 'english.pt'],
    ['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '2', 'dropout', '02', 'wikipedia', 'english.pt'],
    ['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '2', 'dropout', '02', 'Gutenberg_big', '3', 'english.pt'],
    ['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '2', 'dropout', '02', 'Gutenberg_big', '2', 'english.pt'],
    ['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '2', 'dropout', '02', 'Gutenberg_big', '1', 'english.pt'],
    ['LSTM', 'embedding-size', '768', 'nhid', '300', 'nlayers', '1', 'dropout', '02', 'Gutenberg_big_+_wiki', '2', 'english.pt'],
]
names = [
    #'embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_+_wiki_+_lpp_2_english',
    #'embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_+_wiki_2_english',
    #'embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_2_english',
    'embedding-size_768_nhid_768_nlayers_2_dropout_02_wikipedia_english',
    'embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_3_english',
    'embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_2_english',
    'embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_1_english',
    'embedding-size_768_nhid_300_nlayers_1_dropout_02_gutenberg_big_+_wiki_2_english',
]
config_paths = [
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_memory-size-inf_gutenberg_big_+_wiki_+_lpp_2_english.yml',
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_memory-size-inf_gutenberg_big_+_wiki_2_english.yml',
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_memory-size-inf_gutenberg_big_2_english.yml',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_memory-size-inf_wikipedia_english.yml',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_memory-size-inf_gutenberg_big_3_english.yml',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_memory-size-inf_gutenberg_big_2_english.yml',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_memory-size-inf_gutenberg_big_1_english.yml',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-300_nlayers-1_dropout-02_memory-size-inf_gutenberg_big_+_wiki_2_english.yml',
]
saving_path_folders = [
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_gutenberg_big_+_wiki_+_lpp_2',
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_gutenberg_big_+_wiki_2',
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_gutenberg_big_2',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_wikipedia',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_gutenberg_big_3',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_gutenberg_big_2',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_gutenberg_big_1',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size-768_nhid-300_nlayers-1_dropout-02_gutenberg_big_+_wiki_2',
]
prediction_types = [
    #'sequential',
    #'sequential',
    #'sequential',
    'sequential',
    'sequential',
    'sequential',
    'sequential',
    'sequential',
]
pretrained_lstm_models = [
    trained_model_folder + name_templates[index].format(rnn_type, ninp, nhid, nlayer, dropout, memory_size) for index, (rnn_type, ninp, nhid, nlayer, dropout, memory_size) in enumerate(zip(rnn_types, ninps, nhids, nlayers, dropouts, memory_sizes))] # path to the model from which we want to retrieve the activations


In [9]:
#pretrained_lstm_models = [
#    trained_model_folder + name_templates[index].format(rnn_type, ninp, nhid, nlayer, dropout, memory_size) for index, (rnn_type, ninp, nhid, nlayer, dropout, memory_size) in enumerate(zip(rnn_types, ninps, nhids, nlayers, dropouts, memory_sizes))] # path to the model from which we want to retrieve the activations
#infos = [os.path.basename(model).split('_') for model in pretrained_lstm_models]
#names = ['_'.join(os.path.basename(model).split('.')[0].split('_')[1:]) for model in pretrained_lstm_models]
#config_paths = [os.path.join(config_path_folder, 'config_' + name + '.yml') for name in names]
#config_paths = ['_'.join(config.split('_')[:-3]) + '_' + '_'.join(config.split('_')[-2:]) for config in config_paths]
#saving_path_folders = [
#    os.path.join(path_to_data, '{}/{}_Gutenberg_1'.format(language, name)) for name in names]
#prediction_types = ['sequential' for i in pretrained_lstm_models]

In [10]:
for path in pretrained_lstm_models:
    print(path)
    print(os.path.exists(path))

/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_wikipedia_english.pt
True
/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_3_english.pt
True
/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_2_english.pt
True
/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_1_english.pt
True
/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_embedding-size_768_nhid_300_nlayers_1_dropout_02_gutenberg_big_+_wiki_2_english.pt
True


In [7]:
def get_config(rnn_type='LSTM',
               language='english',
               ntoken=50001,
               ninp=650,
               nhid=650,
               nlayers=2,
               dropout='02',
               memory_size=np.inf,
               tie_weights=False,
               eos_separator='<eos>',
               cuda=True,
               weights_path=None,
               path_to_vocab=None,
               includ_surprisal=True,
               includ_entropy=True,
               parameters=['in', 'forget', 'out', 'c_tilde', 'hidden', 'cell']):
    config_template = {
        'rnn_type': rnn_type,
        'language': language,
        'ntoken': ntoken,
        'ninp': ninp,
        'nhid': nhid,
        'nlayers': nlayers,
        'dropout': int(dropout)/10,
        'memory_size': memory_size,
        'tie_weights': tie_weights,
        'eos_separator': eos_separator,
        'cuda': cuda,
        'weights_path': os.path.join(weights_path, name_template.format(rnn_type, ninp, nhid, nlayers, dropout, 0)).replace('_memory-size-0', ''),
        'path_to_vocab': path_to_vocab,
        'includ_surprisal': includ_surprisal,
        'includ_entropy': includ_entropy,
        'parameters': parameters}
    return config_template


In [33]:
for index, (rnn_type, ninp, nhid, nlayer, dropout, memory_size) in enumerate(zip(rnn_types, ninps, nhids, nlayers, dropouts, memory_sizes)):
    config_template =  get_config(rnn_type=rnn_type,
                                   language='english',
                                   ntoken=50001,
                                   ninp=int(ninp),
                                   nhid=int(nhid),
                                   nlayers=int(nlayer),
                                   dropout=dropout,
                                   memory_size=memory_size,
                                   weights_path=trained_model_folder,
                                   path_to_vocab=vocab_path,
                                 parameters=['hidden'])
    save_yaml(config_template, config_paths[index])

In [34]:
config_template

{'rnn_type': 'LSTM',
 'language': 'english',
 'ntoken': 50001,
 'ninp': 650,
 'nhid': 650,
 'nlayers': 2,
 'dropout': 0.2,
 'memory_size': 100,
 'tie_weights': False,
 'eos_separator': '<eos>',
 'cuda': True,
 'weights_path': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/weights_LSTM_embedding-size-650_nhid-650_nlayers-2_dropout-02_wiki-kristina_english.pt',
 'path_to_vocab': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training',
 'includ_surprisal': True,
 'includ_entropy': True,
 'parameters': ['hidden']}

Creating iterator for each run:

In [119]:
acc = 0
for i in iterator_list:
    for j in i:
        if 'unk' in j:
            acc+=1
            print(j)

<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk

<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk

<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk

In [120]:
acc

4035

In [15]:
os.path.basename(paths[0])[:-3]

'LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_Gutenberg_big_+_wiki_2_english_for_RSA_checkpoint_epoch-1_batch-0.'

## Activation extraction

In [27]:
paths = [
#"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_L-4_H-768_final/LSTM_embedding-size_768_nhid_768_nlayers_4_dropout_02_final_version_englishfinal_version.pt",
#"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_L-2_H-768_final/LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_final_version_englishfinal_version.pt",
#"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_L-1_H-768_final/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_final_version_englishfinal_version.pt",
#"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_L-1_H-768_final/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_final_version_englishfinal_version_checkpoint_epoch-10_batch-0.pt",



#"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_L-2_H-768_xlarge/LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_Gutenberg_xlarge_english.pt",
#"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_L-1_H-768_wikipedia/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_wikipedia_english.pt",
#"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_L-2_H-650_wikipedia/hidden650_batch128_dropout0.2_lr20.0.pt",
"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM/LSTM_L-1_H-768_xlarge/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_Gutenberg_xlarge_english.pt",



]

config_paths = [
    #{'cuda': False,'dropout': 0.2,'eos_separator': '<eos>','includ_entropy': True,'includ_surprisal': True,'language': 'english','memory_size': np.inf,'nhid': 768,'ninp': 768,'nlayers': 2,'ntoken': 50001,'parameters': ['hidden'],'path_to_vocab': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_xlarge','rnn_type': 'LSTM','tie_weights': False,'weights_path': paths[0]},
    #{'cuda': False,'dropout': 0.2,'eos_separator': '<eos>','includ_entropy': True,'includ_surprisal': True,'language': 'english','memory_size': np.inf,'nhid': 768,'ninp': 768,'nlayers': 1,'ntoken': 50001,'parameters': ['hidden'],'path_to_vocab': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/wikipedia','rnn_type': 'LSTM','tie_weights': False,'weights_path': paths[1]},
    #{'cuda': False,'dropout': 0.2,'eos_separator': '<eos>','includ_entropy': True,'includ_surprisal': True,'language': 'english','memory_size': np.inf,'nhid': 650,'ninp': 650,'nlayers': 2,'ntoken': 50001,'parameters': ['hidden'],'path_to_vocab': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/LSTM','rnn_type': 'LSTM','tie_weights': False,'weights_path': paths[5]},
    {'cuda': False,'dropout': 0.2,'eos_separator': '<eos>','includ_entropy': True,'includ_surprisal': True,'language': 'english','memory_size': np.inf,'nhid': 768,'ninp': 768,'nlayers': 1,'ntoken': 50001,'parameters': ['hidden'],'path_to_vocab': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_xlarge','rnn_type': 'LSTM','tie_weights': False,'weights_path': paths[0]},
    ]


rnn_types = ['LSTM'] * len(paths)
ninps = ['768'] * len(paths)
nhids =  ['768'] * len(paths)
nlayers = ['1'] * 3
dropouts =  ['02'] * len(paths)
memory_sizes = [np.inf] * len(paths) #, np.inf
vocab_paths = [config['path_to_vocab'] for config in config_paths]
config_path_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/'
path_to_data = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/'
#names = [os.path.basename(path)[:-3].replace('dropout_02_final_version_englishfinal_version_checkpoint', 'trained') for path in paths]
names = [
#"LSTM_L-2_H-768_xlarge_trained",
#"LSTM_embedding-size-768_nhid-768_nlayers-1_dropout-02_wikipedia",
#"LSTM_L-2_H-650_wikipedia_trained",
"LSTM_L-1_H-768_xlarge_trained",
]
saving_path_folders = [f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/{name}' for name in names]
#names = ["LSTM_embedding-size_768_nhid_768_nlayers_4_seed-1111", "LSTM_embedding-size_768_nhid_768_nlayers_2_seed-1111", "LSTM_embedding-size_768_nhid_768_nlayers_1_seed-1111"]
#saving_path_folders = [f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/{name}' for name in names]
prediction_types = ['sequential'] * len(paths)


In [28]:
for index, config in enumerate(config_paths):
    # tokenizing LPP text
    paths = sorted(glob.glob(template))
    vocab_path = vocab_paths[index]
    vocab = Dictionary(vocab_path, language)
    iterator_list = [tokenize(path, language, train=False, vocab=vocab) for path in paths]

    # instanciating LSTM extractor
    extractor = LSTMExtractor(config, 
                              language, 
                              names[index], 
                              prediction_types[index], 
                              output_hidden_states=True, 
                              memory_size=memory_sizes[index], 
                              randomize=False,
                              seed=1111
                             )

    # Extracting activations
    print(extractor.name, ' - Extracting activations ...')
    for run_index, iterator in tqdm(enumerate(iterator_list)):
        print("############# Run {} #############".format(run_index))
        activations  = extractor.extract_activations(iterator, language)

        transform(
            activations, 
            saving_path_folders[index], 
            'activations', 
            run_index=run_index,
            n_layers_hidden=int(nlayers[index]),
            hidden_size=int(nhids[index]))  
        #activations.to_csv(os.path.join(saving_path_folders[index], 'activations_run{}.csv'.format(run_index + 1)), index=False)




100%|██████████| 207/207 [00:01<00:00, 150.88it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
7it [00:00, 62.90it/s]

LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02  - Extracting activations ...
############# Run 0 #############



15it [00:00, 65.28it/s]
23it [00:00, 67.02it/s]
31it [00:00, 68.30it/s]
39it [00:00, 69.22it/s]
47it [00:00, 69.99it/s]
55it [00:00, 70.63it/s]
63it [00:00, 71.06it/s]
71it [00:01, 71.39it/s]
79it [00:01, 71.30it/s]
86it [00:01, 70.49it/s]
94it [00:01, 70.61it/s]
102it [00:01, 70.88it/s]
110it [00:01, 71.44it/s]
118it [00:01, 71.86it/s]
126it [00:01, 71.89it/s]
134it [00:01, 72.19it/s]
142it [00:01, 72.44it/s]
150it [00:02, 72.61it/s]
158it [00:02, 72.76it/s]
166it [00:02, 72.82it/s]
174it [00:02, 72.88it/s]
182it [00:02, 72.85it/s]
190it [00:02, 72.45it/s]
198it [00:02, 72.60it/s]
206it [00:02, 72.69it/s]
214it [00:02, 72.80it/s]
222it [00:03, 72.85it/s]
230it [00:03, 72.83it/s]
238it [00:03, 72.01it/s]
246it [00:03, 71.32it/s]
254it [00:03, 71.55it/s]
262it [00:03, 71.77it/s]
270it [00:03, 71.80it/s]
278it [00:03, 71.94it/s]
286it [00:03, 72.03it/s]
294it [00:04, 72.15it/s]
302it [00:04, 72.21it/s]
310it [00:04, 72.29it/s]
318it [00:04, 72.31it/s]
326it [00:04, 72.39it/s]
334it [00:

(1894, 768)
(1894, 770)


1it [00:30, 30.97s/it]
0it [00:00, ?it/s]
3it [00:00, 29.50it/s]

...

############# Run 8 #############



11it [00:00, 34.03it/s]
19it [00:00, 40.27it/s]
26it [00:00, 45.87it/s]
33it [00:00, 50.93it/s]
40it [00:00, 55.16it/s]
47it [00:00, 58.68it/s]
54it [00:00, 61.45it/s]
61it [00:00, 63.56it/s]
68it [00:01, 65.11it/s]
75it [00:01, 66.22it/s]
82it [00:01, 66.96it/s]
89it [00:01, 67.77it/s]
96it [00:01, 68.38it/s]
103it [00:01, 68.77it/s]
110it [00:01, 69.00it/s]
117it [00:01, 69.15it/s]
124it [00:01, 69.29it/s]
131it [00:01, 69.31it/s]
138it [00:02, 68.20it/s]
145it [00:02, 67.96it/s]
152it [00:02, 68.39it/s]
159it [00:02, 68.73it/s]
166it [00:02, 68.99it/s]
173it [00:02, 69.25it/s]
180it [00:02, 69.43it/s]
187it [00:02, 69.50it/s]
194it [00:02, 69.51it/s]
201it [00:02, 69.48it/s]
208it [00:03, 69.51it/s]
215it [00:03, 69.53it/s]
222it [00:03, 69.62it/s]
229it [00:03, 69.73it/s]
236it [00:03, 69.76it/s]
243it [00:03, 69.81it/s]
250it [00:03, 69.58it/s]
257it [00:03, 69.62it/s]
264it [00:03, 69.69it/s]
271it [00:03, 69.70it/s]
278it [00:04, 69.70it/s]
285it [00:04, 68.95it/s]
292it [00:04

2021it [00:29, 69.59it/s]
2028it [00:29, 69.65it/s]
2035it [00:29, 69.64it/s]
2042it [00:29, 69.73it/s]
2049it [00:29, 69.35it/s]
2056it [00:29, 68.95it/s]
2063it [00:29, 68.15it/s]
2070it [00:29, 67.56it/s]
2077it [00:30, 68.06it/s]
2084it [00:30, 68.58it/s]
2091it [00:30, 68.93it/s]
2098it [00:30, 69.10it/s]
2105it [00:30, 69.22it/s]
2112it [00:30, 69.36it/s]
2119it [00:30, 69.39it/s]
2126it [00:30, 69.34it/s]
2133it [00:30, 69.30it/s]
2140it [00:30, 69.27it/s]
2147it [00:31, 69.25it/s]
2154it [00:31, 69.31it/s]
2161it [00:31, 69.05it/s]
2168it [00:31, 69.13it/s]
2175it [00:31, 69.23it/s]
2182it [00:31, 69.11it/s]
2189it [00:31, 69.17it/s]
2196it [00:31, 69.29it/s]
2203it [00:31, 69.38it/s]
2210it [00:32, 68.96it/s]
2217it [00:32, 67.97it/s]
2224it [00:32, 68.13it/s]
2231it [00:32, 68.43it/s]
2238it [00:32, 68.63it/s]
2245it [00:32, 68.84it/s]
2252it [00:32, 69.03it/s]
2259it [00:32, 69.13it/s]
2266it [00:32, 69.18it/s]
2273it [00:32, 69.03it/s]
2280it [00:33, 69.15it/s]
2287it [00:3

(2530, 768)
(2530, 770)


9it [05:32, 36.97s/it]


## Creating datasets for training

In [18]:
import lstm_utils as utils
import random
from joblib import Parallel, delayed

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
def clean(sent):
    if len(sent.replace('<unk>', '')) > 7:
        if (sent.count('<unk>') / len(sent.split(' '))) <0.10:
            return sent
        
def f(text):
    return ' '.join(tokenize(text, 'english', True, vocab=None))

def create_set(block, saving_path, relative_size=30, seed=1111, name='test'):
    """Create test set from block with given proportion (relative_size).
    """
    random.seed(seed)
    random.shuffle(block)
    N = len(block)
    assert ((relative_size<=100) and (relative_size>0))
    test_set = block[:int(N*relative_size/100)]
    preprocessed_texts = Parallel(n_jobs=-1)(delayed(f)(text) for text in tqdm(test_set))
    test_text = ' '.join(preprocessed_texts)
    
    tmp = test_text.split('<eos>\n')
    test_text = '<eos>\n'.join(list(filter(None, Parallel(n_jobs=-1)(delayed(clean)(sent) for sent in tqdm(tmp)))))

    utils.write(os.path.join(saving_path, f'{name}.txt'), test_text)



In [24]:

seed = 1111
files = sorted(glob.glob('/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/Gutenberg/books/*_corrected.txt'))

random.seed(seed)
random.shuffle(files)

blocks = list(chunks(files, 3813*3)) # 3813*3 books for each LSTM --> 3 datasets are created
# [['B', 'H', 'G'], ['D', 'A', 'C'], ['E', 'F', 'I'], ['J', 'K']]

In [35]:
l1 = 31*41
print(l1)
l2 = 31*41*3
print(l2)
l3 = 31*41*3*3
print(l3)
l4 = 31*41*2*3*3
print(l4)

1271
3813
11439
22878


In [38]:

joined_block = blocks[0]+blocks[1]
names = ["Gutenberg_small", "Gutenberg_medium", "Gutenberg_large", "Gutenberg_xlarge"]

for index, l in enumerate([l1, l2, l3, l4]):
    block = joined_block[:l]
    preprocessed_texts = Parallel(n_jobs=-1)(delayed(f)(text) for text in tqdm(block))
    block_text = ' '.join(preprocessed_texts)

    tmp = block_text.split('<eos>\n')
    block_text = '<eos>\n'.join(list(filter(None, Parallel(n_jobs=-1)(delayed(clean)(sent) for sent in tqdm(tmp)))))


    utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/{names[index]}/train.txt', block_text)
    #block_text += wikipedia_train
    #utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_+_wiki_{index+1}/train.txt', block_text)
    #block_text += lpp
    #utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_+_wiki_+_lpp_{index+1}/train.txt', block_text)


100%|██████████| 47395788/47395788 [26:58<00:00, 29290.43it/s]


In [6]:
wikipedia_train = '/Volumes/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/wikipedia/train.txt'
wikipedia_train = open(wikipedia_train, 'r').read()

paths = sorted(glob.glob(template))
iterator_list = [tokenize(path, language, train=False, vocab=None) for path in paths]
lpp = ' '.join([' '.join(iterator_list[i]).replace('<eos>', '<eos>\n') for i in range(9)])

100%|██████████| 207/207 [00:00<00:00, 164404.64it/s]


In [7]:

for index, block in enumerate(blocks):
    preprocessed_texts = Parallel(n_jobs=-1)(delayed(f)(text) for text in tqdm(block))
    block_text = ' '.join(preprocessed_texts)
    
    tmp = block_text.split('<eos>\n')
    block_text = '<eos>\n'.join(list(filter(None, Parallel(n_jobs=-1)(delayed(clean)(sent) for sent in tqdm(tmp)))))

    #utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_{index+1}/train.txt', block_text)

    utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_{index+1}/train.txt', block_text)
    #block_text += wikipedia_train
    #utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_+_wiki_{index+1}/train.txt', block_text)
    #block_text += lpp
    #utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_+_wiki_+_lpp_{index+1}/train.txt', block_text)


 15%|█▌        | 576/3813 [00:09<00:31, 101.34it/s]/Applications/anaconda3/envs/parietal/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
100%|██████████| 8059516/8059516 [03:36<00:00, 37281.26it/s]


In [8]:
for index, _ in enumerate(blocks):
    saving_path = f'/Volumes/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_{index+1}'
    create_set(blocks[(index+1)%len(blocks)], saving_path, relative_size=10, seed=1111, name='test')
    create_set(blocks[(index+2)%len(blocks)], saving_path, relative_size=10, seed=1111, name='valid')
    

100%|██████████| 738149/738149 [00:18<00:00, 38909.49it/s]


### Creating vocabulary files

In [39]:
folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training'
language = 'english'
datasets = ["Gutenberg_small", "Gutenberg_medium", "Gutenberg_large", "Gutenberg_xlarge"]

In [40]:
for dataset in tqdm(datasets):
    path = os.path.join(folder, dataset)
    dico = Dictionary(path, language)
    assert len(dico)==len(dico.word2idx.keys())
    assert len(dico)==len(dico.idx2word)

  0%|          | 381978/159344695 [00:00<01:24, 1887807.18it/s]


 34%|███▎      | 53614408/159344695 [00:24<00:47, 2237703.62it/s]


 66%|██████▋   | 105821778/159344695 [00:47<00:24, 2186048.65it/s]


 99%|█████████▊| 157241295/159344695 [01:11<00:00, 2183624.88it/s]


  9%|▉         | 45497487/481715698 [00:23<03:40, 1979120.45it/s]


...

 81%|████████  | 770801905/954997913 [05:57<01:26, 2134853.73it/s]


 86%|████████▌ | 817130006/954997913 [06:21<01:13, 1888575.15it/s]


 90%|█████████ | 863498418/954997913 [06:44<00:46, 1973915.14it/s]


 95%|█████████▌| 909793471/954997913 [07:08<00:22, 1972927.34it/s]


100%|██████████| 4/4 [22:28<00:00, 337.09s/it]
